In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn as nn
from receptivefield.pytorch import PytorchReceptiveField
from receptivefield.image import get_default_image
from lib import autoencoder
from torchvision import models
import torch
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
# Input shape needs to be large as the receptive field is usually quite
# large for the architectures used. As the package computes the RF 
# numerically we need to make sure that it can fit within the 
# input shape
input_shape = [1024, 1024, 3] 

In [4]:
class ModelWrapper(nn.Module):
    def __init__(self):
        super(ModelWrapper, self).__init__()
        #model = models.resnet50()
        #print(model)
        #self.autoencoder = torch.nn.Sequential(*(list(model.children())[:5]))
        self.autoencoder = autoencoder.FeatureEncoder()
        #self.autoencoder.eval()
    def forward(self, x):
        y = self.autoencoder(x, for_receptive_field=True)
        self.feature_maps = [
          y['early'],
          y['middle'],
          y['deep']
        ]
        #self.feature_maps = [y]
        return y
def model_fn():
    model = ModelWrapper()
    model.eval()
    return model

In [5]:
model = model_fn()

In [6]:
img = get_default_image((1024, 1024,3), name='cat')
print(img.shape)
print(model(torch.from_numpy(np.random.normal(size=(1,3,224,224)).astype(np.float32)))['deep'].size())

(1024, 1024, 3)
torch.Size([1, 16, 28, 28])


In [7]:
# Debug code
# import torch
# import numpy as np
# import torchvision.models
# model = model_fn()
# model(torch.from_numpy(np.random.normal(size=(1,3,224,224)).astype(np.float32)))

In [8]:
rf = PytorchReceptiveField(model_fn)
rf_params = rf.compute(input_shape = input_shape)
# plot receptive fields
rf.plot_rf_grids(get_default_image(input_shape, name='cat'))

[2020-12-23 18:46:59,872][ INFO][pytorch.py]::Feature maps shape: [GridShape(n=1, w=128, h=128, c=16), GridShape(n=1, w=128, h=128, c=16), GridShape(n=1, w=128, h=128, c=16)]
[2020-12-23 18:46:59,873][ INFO][pytorch.py]::Input shape       : GridShape(n=1, w=1024, h=1024, c=3)
/home/mark/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


AttributeError: 'NoneType' object has no attribute 'detach'